# Lab 05


In [5]:
%%capture
!wget https://github.com/dbdmg/data-science-lab/raw/master/datasets/NYC_POIs.zip -O nyc_poi.zip
!wget https://github.com/dbdmg/data-science-lab/raw/master/datasets/US_FlightDelayData.zip -O fdd.zip

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## New York Point Of Interest

The _New York Point Of Interest_ dataset collects a sub-sample of the point of interests (POI) available in
the city of New York. It contains the coordinates of each point of interest and the category to which it
belongs to, specifying the _type_ of the POI, for each POI _category_. There are _four_ possible categories of POIs:
_amenity, shop, public_transport_ and _highway_.
The fields available in the dataset are:
- `@id`: a unique id for each point of interest.
- `@lat`: latitude coordinate of the POI in decimal degrees.
- `@lon`: longitude coordinate of the POI in decimal degrees.
- `amenity name`: if the POI category is amenity its type is reported in this field.
- `shop`: if the POI category is shop its type is reported in this field.
- `public_transport`: if the POI category is public transport its type is reported in this field.
- `highway`: if the POI category is highway its type is reported in this field.
  
To identify which are the POIs belonging only to the New York City municipality, a further file is provided. The _New York City municipality POIs metadata_ file is composed by a single column containing the Ids of the POI of interest of the municipality.

In [7]:
from zipfile import ZipFile

with ZipFile('nyc_poi.zip', 'r') as zip:
    zip.printdir()

File Name                                             Modified             Size
New_York_City_Map.PNG                          2019-11-07 13:54:28        87105
__MACOSX/._New_York_City_Map.PNG               2019-11-07 13:54:28          211
ny_municipality_pois_id.csv                    2019-11-07 15:50:52       581601
__MACOSX/._ny_municipality_pois_id.csv         2019-11-07 15:50:52          267
pois_all_info                                  2019-11-07 14:56:46      4787894
__MACOSX/._pois_all_info                       2019-11-07 14:56:46          267


In [15]:
files_to_extract = ['New_York_City_Map.PNG', 'ny_municipality_pois_id.csv', 'pois_all_info']
with ZipFile('nyc_poi.zip', 'r') as zip:
    for file in files_to_extract:
        zip.extract(file)

In [16]:
nyc_df = pd.read_csv('pois_all_info', sep='\t')

/var/folders/2q/7qslh1r50qv5dxqpmmm9jcy00000gn/T/ipykernel_1732/1444405744.py:1: DtypeWarning: Columns (4,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  nyc_df = pd.read_csv('pois_all_info', sep='\t')


In [17]:
nyc_df.head()

,@id,@type,@lat,@lon,amenity,name,shop,public_transport,highway
0,42432939,node,40.814104,-73.949623,NaN,NaN,NaN,stop_position,NaN
1,42448838,node,40.761647,-73.949864,NaN,NaN,NaN,stop_position,NaN
2,42723103,node,40.852182,-73.772677,ferry_terminal,Hart Island Ferry Terminal,NaN,NaN,NaN
3,42764076,node,40.867164,-73.882158,NaN,Botanical Garden,NaN,stop_position,NaN
4,42811266,node,40.704806,-73.772734,NaN,NaN,NaN,stop_position,NaN


## Flight Delay Data

Measuring the performance of flight carriers (e.g. American Airlines, EasyJet) is extremely important for
the transportation department and, for this reason, all the information related to each flight are constantly
monitored and collected in huge databases by the Department of Transportation. To the aim of this laboratory, just a small set of information has been extracted.

The dataset contains the Carrier On-Time Performance information collected from 01-01-2017 until 31-01-2017 for all the flights in the United States. Each row represents a flight in a specific day.

Some of the most useful fields in the dataset are:
- `FL_DATE`: day of the flight in format YYYY-mm-dd.
- `TAIL_NUM`: aircraft registration number, unique to a single aircraft.
- `UNIQUE_CARRIER`: flight carrier id.
- `FL_NUM`: number of the flight.
- `ORIGIN`: departure airport code.
- `DEST`: destination airport code.
- `CRS_DEP_TIME`: scheduled departure time (local time: HHMM) shown in the carriers’ Computerized
Reservations Systems (CRS)
- `DEP_TIME`: actual departure time (local time: HHMM)
- `DEP_DELAY`: overall delay at departure. Difference in minutes (floating point number) between sched-
uled and actual departure time. Early departures set to 0.
- `CRS_ARR_TIME`: scheduled arrival time (local time: HHMM) shown in the carriers’ Computerized
Reservations Systems (CRS)
- `ARR_TIME`: actual arrival time (local time: HHMM)
- `ARR_DELAY`: overall delay. Difference in minutes (floating point number) between scheduled and
actual arrival time. Early arrivals show negative numbers.
- `CARRIER_DELAY`: delay in minutes (floating point number) caused by the carrier.
- `WEATHER_DELAY`: delay in minutes (floating point number) caused by the weather.
- `NAS_DELAY`: delay in minutes (floating point number) caused by the National Air System (NAS).
- `SECURITY_DELAY`: delay in minutes (floating point number) caused by the security.
- `LATE_AIRCRAFT_DELAY`: delay in minutes (floating point number) caused by the aircraft.

There are some other fields in this dataset. You can explore them, understand what they represents and
whether they are significant or not for your analysis.

In [18]:
from zipfile import ZipFile

with ZipFile('fdd.zip', 'r') as zip:
    zip.printdir()

File Name                                             Modified             Size
831394006_T_ONTIME.csv                         2019-12-10 05:10:20     92160796


In [19]:
with ZipFile('fdd.zip', 'r') as zip:
    zip.extractall()

In [21]:
fdd_df = pd.read_csv('831394006_T_ONTIME.csv')
fdd_df.head()

,FL_DATE,UNIQUE_CARRIER,AIRLINE_ID,TAIL_NUM,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,...,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 32
0,2017-01-01,AA,19805,N787AA,1,12478,1247803,31703,JFK,"New York, NY",...,1209.0,27.0,0.0,NaN,27.0,0.0,0.0,0.0,0.0,NaN
1,2017-01-01,AA,19805,N783AA,2,12892,1289204,32575,LAX,"Los Angeles, CA",...,1809.0,42.0,0.0,NaN,34.0,0.0,8.0,0.0,0.0,NaN
2,2017-01-01,AA,19805,N791AA,4,12892,1289204,32575,LAX,"Los Angeles, CA",...,2040.0,42.0,0.0,NaN,7.0,0.0,0.0,0.0,35.0,NaN
3,2017-01-01,AA,19805,N391AA,5,11298,1129804,30194,DFW,"Dallas/Fort Worth, TX",...,1749.0,97.0,0.0,NaN,77.0,0.0,20.0,0.0,0.0,NaN
4,2017-01-01,AA,19805,N346AA,6,13830,1383002,33830,OGG,"Kahului, HI",...,642.0,42.0,0.0,NaN,0.0,0.0,42.0,0.0,0.0,NaN
